# Foreward

*Let the compiler teach you 64-bit x86 assembler, one example at a time.*

Whenever I've helped some get started with assembler, whether in person or via a forum or message board I've always given the same advice regarding [Matt Godbolt's Compiler Explorer](https://godbolt.org): learn from the compiler.

I stand by this advice 100%. However I also suspect that following it is not easy. It can take a lot of work to figure out how to get the compiler to help you! On that basis I decided to throw together an introduction to x86-64 course (x86-64 being something I needed to learn). The course consists almost entirely of assembler examples generated by gcc.

What does that actually look like? Well, the course is full of little boxes full of C code (like the one below). Each one is followed by the resulting assembly output.

In [1]:
%%python -m gcc
int square(int x)
{
    return x * x;
}

square:
	imul	edi, edi
	mov	eax, edi
	ret


Examples like the above are compiled at `-O2` unless otherwise stated.

Whilst we might, one day, outgrow the teachings of the compiler, it can give us a **lot** of help along the way.
For example, I have a strong interest in machine level debugging, but most of my expertise and experience is on RISC architectures.
I'm confident the compiler can teach me (almost) everything I need to know about assembly level debugging simply because (almost) everything I need to debug will have been issued by the compiler.

The gaps the compiler leaves are usually in the fine details of hand-writing assembler.
Good assembly authors know how to use the assembler's macro system to minimize repetition. 
Macros are a great example of something the compiler will not teach us.
It's impossible to learn macros from the compiler because it never generates them.

Regardless of whether you might one day need to learn a few things beyond the scope of the compiler (and therefore of this book), it's still worth buckling in and seeing just how much assembler the compiler can teach us!

# Introduction

## AT&T versus Intel syntax

Before we start it's probably useful to mention that, unlike almost every other architecture that is remains relevant in the 21st century, x86 supports multiple assembly syntaxes.

x86 software that originates from the Unix tradition typically uses AT&T syntax.
That shouldn't be a surprise to anyone since that's where Unix was born!
It also means that all x86 assembly in the Linux kernel sources uses AT&T syntax and it is the default for GNU tools such as gcc and gdb.
AT&T has distinctive for the following:

* Uses `%` as a sigil for register names
* Has destination-last operand ordering (e.g. `imull %esi, %edi` ⇒ `edi += esi`)
* Demarks addressing modes with (parentheses)
* Has explicit operand widths (`addl` operates on 32-bit "long" values)

gcc will use AT&T by default (including for inline assembly).
It can also be be explicitly requested with `gcc -masm=att`:

In [16]:
%%python -m gcc -masm=att
int add_sub(int a, int b, int c, int d)
{
    return a + b + c - d;
}

add_sub:
	addl	%esi, %edi
	leal	(%rdi,%rdx), %eax
	subl	%ecx, %eax
	ret


For code originating in the DOS/Windows tradition then Intel syntax is more common.
Intel syntax gets is name because it's the format used in the Intel documentation.
Intel syntax:

* Does not use sigils for register names
* Has destination-first operand ordering
* Dermarks addressing modes with [square brackets]
* Infers operand widths from the register names (`add` operates on 32-bit long value because the opperand is `edi` which is the 32-bit view of the `rdi`/`edi`/`di`/`dl` register)

gcc will use Intel syntax if requested with `-masm=intel`:

In [18]:
%%python -m gcc -masm=intel
int add_sub(int a, int b, int c, int d)
{
    return a + b + c - d;
}

add_sub:
	add	edi, esi
	lea	eax, [rdi+rdx]
	sub	eax, ecx
	ret


Intel syntax, with it's destination on the left design and the use of [square brackets] for its addressing mode operands, is arguably **much** closer to the idioms used in modern RISC assembly languages (including those both Arm and RISC-V).

In order to help those with a strong RISC background, this document uses, Intel assembly despite the author's strong affiliation to the Unix traditions!
If you prefer to work with AT&T assembly, then go to Appendix A and change `SYNTAX` to `-masm=intel` and regenerate the document (twice).

## x86-64 registers

For now, let's ignore the floating point and vector registers and focus only on the general-purpose integer registers.
These are the registers we need to learn the base instruction set will focus on. 

x86-64 has 16 mostly general purpose registers.
The "mostly" appears for a couple of reasons: rsp is the stack pointer and seldom used for anything truely general purpose and some instructions implicitly use specific named registers.
Nevertheless the general purpose register do have much in common.
All registers are capable of holding 64-bit integers and each register has multiple names, where the specific name describes the access width.

The register set splits naturally into two parts.
The named registers are inherited from the legacy 32-bit x86 whilst the numbered register were introduced specifically for x86-64.
All names registers use the same pattern to access the 64- and 32-bit variants and all access to numbers registers are consistant.

This summary of the register set shows named registers on the left and numbered registers on the right:

```
[---------- 64-bit -----------]
             [---- 32-bit ----]
                    [- 16-bit ]
                         [ 8b ]

+------------+------+----+----+    +------------+------+----+----+
|    rax     | eax  | ax | al |    |     r8     |  r8d | r8w| r8b|
|    rbx     | ebx  | bx | bl |    |     r9     |  r9d | r9w| r9b|
|    rcx     | ecx  | cx | cl |    |    r10     | r10d |r10w|r10b|
|    rdx     | edx  | dx | dl |    |    r11     | r11d |r11w|r11b|
|    rsi     | esi  | si |sil |    |    r12     | r12d |r12w|r12b|
|    rdi     | edi  | di |dil |    |    r13     | r13d |r13w|r13b|
|    rbp     | ebp  | bp |bpl |    |    r14     | r14d |r14w|r14b|
|    rsp     | esp  | sp |spl |    |    r15     | r15d |r15w|r15b|
+------------+------+----+----+    +------------+------+----+----+
```
<center>*Table: 16 General Purpose registers (including the stack pointer)*</center>

This collection of functions below takes a single argument and returns it as a 64-bit value. This demonstrates moving each of the different views of the same register (`rdi`/`edi`/`di`/`dil`) to `rax`.

In [4]:
%%python -m gcc
#include <stdint.h>
int64_t f_int64(int64_t x) { return x; }
int64_t f_int32(int32_t x) { return x; }
int64_t f_int16(int16_t x) { return x; }
int64_t f_int8(int8_t x) { return x; }

f_int64:
	mov	rax, rdi
	ret
f_int32:
	movsxd	rax, edi
	ret
f_int16:
	movsx	rax, di
	ret
f_int8:
	movsx	rax, dil
	ret


# Improbable idioms

At this stage in the course we need to introduce a couple of quirky compiler features.
If this was a "normal" assembler course then this material would appear near the end, or maybe even not appear at all.
However we need to cover this material now because we are planning to let the compiler teach us things.
That means we need to recognize these improbable idioms because the compiler is going to keep showing them to us!



## Generating constant zero

Let's keep things simple and look at the code generated to load the constant zero:

In [5]:
%%python -m gcc
int zero(void) {
    return 0;
}

zero:
	xor	eax, eax
	ret


Weird, it's generating an xor! Most arithmetic operation on x86 take two registers meaning one is both the an operating and the destination. In this case the xor is, effectively, `eax ^= eax`. Regardless of the initial value, eax will be set to zero (and due to zero/sign-extension, rax also becomes zero).

Why?

Take a look a the actual machine code. The xor instruction is just two bytes.

In [6]:
%%python -m gcc objdump
int zero (void) {
    return 0;
}

0000000000000000 <zero>:
   0:	31 c0                	xor    eax,eax
   2:	c3                   	ret


If we compare it to loading the constant one, then we can see a simple `mov` instruction but the instruction sequence if five bytes long. So the compiler prefers `xor` to reduce the code size!

In [7]:
%%python -m gcc objdump
int one (void) {
    return 1;
}

0000000000000000 <one>:
   0:	b8 01 00 00 00       	mov    eax,0x1
   5:	c3                   	ret


## Multiply by constant

In [8]:
%%python -m gcc
long x3(long x) {
    return x * 3;
}

x3:
	lea	rax, [rdi+rdi*2]
	ret


In [9]:
%%python -m gcc
long x17_plus_23(long x) {
    return x * 17 + 23;
}

x17_plus_23:
	mov	rax, rdi
	sal	rax, 4
	lea	rax, [rax+23+rdi]
	ret


We'll talk move about addressing later but for now we'll mention that Intel assembler allows us to ways to express the "plus some constant" addressing mode. gcc typically presents it as `23[rax+rdi]` in the example above. However it can also be expressed entirely within the square brackets: `[rax+rdi+23]`.

TODO: double check this with a build

## Procedure call ABI

TODO

# Addressing modes

In [10]:
%%python -m gcc
struct s { int a; int b; };
void ss(struct s * ptr)
{
    ptr[5].b = 42;
}

void sn(struct s *ptr, unsigned long offset)
{
    ptr[offset].b = 42;
}

ss:
	mov	DWORD PTR [rdi+44], 42
	ret
sn:
	mov	DWORD PTR [rdi+4+rsi*8], 42
	ret


# Appendix A - gcc.py

In [11]:
%%file gcc.py
import subprocess
import sys

# This is correct for Debian (and works nicely on my Arm laptop).
# Free to replace it with plain old "gcc" if you have an x86-64
# computer!
#CROSS_COMPILE = "x86_64-linux-gnu-"
CROSS_COMPILE = ""
CC = CROSS_COMPILE + "gcc"
OBJDUMP = CROSS_COMPILE + "objdump"
SYNTAX = "-masm=intel"

def gcc(program, parameters=[], no_filter=False):
    def filter(ln):
        ln = ln.strip()
        return (
            # Directives we don't care about
            ln.startswith(".arch") or
            ln.startswith(".file") or
            ln.startswith(".text") or
            ln.startswith(".align") or
            ln.startswith(".p2align") or
            ln.startswith(".global") or
            ln.startswith(".globl") or
            ln.startswith(".type") or
            ln.startswith(".intel_syntax") or
            ln.startswith(".size") or
            ln.startswith(".ident") or
            ln.startswith(".section") or
            # "Wildcard" for .cfi_startproc and .cfi_endproc
            ln.startswith(".cfi_") or
            # Labels that are not used as jump targets
            ln.startswith(".LFB") or
            ln.startswith(".LFE")
        )
        
    try:
        result = subprocess.run(
            [CC, SYNTAX, "-O2", "-xc"] +
                parameters +
                ["-S", "-", "-o", "-"],
            input=program,
            capture_output=True,
            text=True,
            check=True,
        )

        asm = [ln for ln in result.stdout.splitlines()
                        if no_filter or not filter(ln)]
        print("\n".join(asm))

    except subprocess.CalledProcessError as e:
        print(e.stderr)
        # Don't print e.stdout, it contains the partially assembled
        # file rather than any useful error output.

def objdump(program, parameters=[], no_filter=False):
    def filter(ln):
        ln = ln.strip()
        return (
            ln == "" or
            ln.startswith("asm.o") or
            ln.startswith("Disassembly")
        )

    try:
        result = subprocess.run(
            [CC, SYNTAX, "-O2", "-xc"] + parameters + ["-c", "-", "-o", "asm.o"],
            input=program,
            capture_output=True,
            text=True,
            check=True,
        )

        result = subprocess.run(
            [OBJDUMP, "-Mintel", "-d", "asm.o"],
            capture_output=True,
            text=True,
            check=True,
        )

        asm = [ln for ln in result.stdout.splitlines() if no_filter or not filter(ln)]
        print("\n".join(asm))

    except subprocess.CalledProcessError as e:
        print(e.stderr)
        print(e.stdout)

if __name__ == "__main__":
    program = sys.stdin.read()
    if len(sys.argv) > 1 and sys.argv[1] == "objdump":
        objdump(program, sys.argv[2:])
    else:
        gcc(program, sys.argv[1:])

Overwriting gcc.py
